In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style
import librosa
import IPython.display
import librosa.display
import os
import random
from matplotlib.pyplot import specgram
import glob

In [2]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    np.nan_to_num(X)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(parent_dir, sub_dirs, file_ext='*.wav'):
    features, labels = np.empty((0, 193)), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            print 'Extracting features for: ', fn
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            features = np.vstack([features,ext_features])
            labels = np.append(labels, os.path.split(fn)[-1].split('-')[1])
            print 'Completed extracting features for: ', fn
            print '\n'
    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [34]:
parent_dir = '../data/UrbanSound8K/audio'
tr_sub_dirs = ['fold1','fold2']
ts_sub_dirs = ['fold3']
# tr_features, tr_labels = parse_audio_files(parent_dir, tr_sub_dirs)
ts_features, ts_labels = parse_audio_files(parent_dir, ts_sub_dirs)

# tr_labels = one_hot_encode(tr_labels)
ts_labels = one_hot_encode(ts_labels)

Extracting features for:  ../data/UrbanSound8K/audio/fold3/90014-8-0-1.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/90014-8-0-1.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/144068-5-4-1.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/144068-5-4-1.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/54898-8-0-10.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/54898-8-0-10.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/146714-0-0-41.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/146714-0-0-41.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/94636-8-0-2.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/94636-8-0-2.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/199769-1-0-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/199769-1-0-0.wav


Extracting feature

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/144068-5-0-1.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-9.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-9.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/165039-7-10-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/165039-7-10-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/107228-5-0-3.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/107228-5-0-3.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-61.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-61.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/30204-0-0-3.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/30204-0-0-3.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/95562-4-0-0.wav
Completed extractin

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/177742-0-0-205.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-38.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-38.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/63292-3-0-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/63292-3-0-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-41.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-41.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/195451-5-0-15.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/195451-5-0-15.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/63095-4-1-4.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/63095-4-1-4.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/63095-4-0-10.wav
Completed extrac

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/63292-3-0-1.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/69598-4-7-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/69598-4-7-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/177742-0-0-49.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/177742-0-0-49.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-63.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-63.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/207124-3-0-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/207124-3-0-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/6988-5-0-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/6988-5-0-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/144068-5-0-12.wav
Completed extracting

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/144068-5-0-9.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/95562-4-3-1.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/95562-4-3-1.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/19338-5-2-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/19338-5-2-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/29721-4-0-2.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/29721-4-0-2.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/74810-9-1-5.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/74810-9-1-5.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/94631-9-1-2.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/94631-9-1-2.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/182103-9-0-53.wav
Completed extracting feat

...

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/186336-9-0-1.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/116400-3-0-2.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/116400-3-0-2.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/199769-1-0-15.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/199769-1-0-15.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/195451-5-0-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/195451-5-0-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/155227-9-0-2.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/155227-9-0-2.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/94636-8-0-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/94636-8-0-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/18594-1-2-0.wav
Completed extract

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/13230-0-0-26.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/159761-0-0-1.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/159761-0-0-1.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/103199-4-2-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/103199-4-2-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/117048-3-0-23.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/117048-3-0-23.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/31884-7-1-4.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/31884-7-1-4.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/195451-5-0-14.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/195451-5-0-14.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/94636-8-0-21.wav
Completed extr

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/179386-3-0-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/94636-8-0-4.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/94636-8-0-4.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-22.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-22.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/42117-8-0-7.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/42117-8-0-7.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-29.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-29.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/42097-7-0-2.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/42097-7-0-2.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/146714-0-0-18.wav
Completed extracting 

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/182103-9-0-26.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/15356-2-0-4.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/15356-2-0-4.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/148838-6-0-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/148838-6-0-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/37560-4-0-2.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/37560-4-0-2.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/13230-0-0-19.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/13230-0-0-19.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/31884-7-0-0.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/31884-7-0-0.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/34708-6-5-0.wav
Completed extracting f

Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/66622-4-0-3.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/76221-2-0-10.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/76221-2-0-10.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-39.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-39.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/195451-5-0-21.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/195451-5-0-21.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-72.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/62837-7-1-72.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-17.wav
Completed extracting features for:  ../data/UrbanSound8K/audio/fold3/22601-8-0-17.wav


Extracting features for:  ../data/UrbanSound8K/audio/fold3/76221-2-0-0.wav
Completed extrac

In [25]:
data_dir = '../data/esc-50'
sample_dir = os.path.join(data_dir, 'sample')
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

print 'Sound Sample Classes'
print '--------------------'
for d in os.listdir(sample_dir):
    print d

Sound Sample Classes
--------------------
rooster
coughing
insects
laughing


In [26]:
samples_dict = dict()
for d in os.listdir(sample_dir):
    sample_class_dir = os.path.join(sample_dir, d)
    samples_dict[d] = [os.path.join(sample_class_dir, f) for f in os.listdir(sample_class_dir)]
print samples_dict

{'insects': ['../data/esc-50/sample/insects/1-46938-A.ogg', '../data/esc-50/sample/insects/1-73585-A.ogg', '../data/esc-50/sample/insects/1-7973-A.ogg', '../data/esc-50/sample/insects/1-17585-A.ogg', '../data/esc-50/sample/insects/1-19501-A.ogg', '../data/esc-50/sample/insects/1-80785-A.ogg', '../data/esc-50/sample/insects/1-46938-B.ogg', '../data/esc-50/sample/insects/1-75189-A.ogg'], 'coughing': ['../data/esc-50/sample/coughing/5-178997-A.ogg', '../data/esc-50/sample/coughing/4-169726-A.ogg'], 'laughing': ['../data/esc-50/sample/laughing/3-118487-A.ogg', '../data/esc-50/sample/laughing/5-263775-B.ogg'], 'rooster': ['../data/esc-50/sample/rooster/1-40730-A.ogg', '../data/esc-50/sample/rooster/1-43382-A.ogg', '../data/esc-50/sample/rooster/1-39923-A.ogg', '../data/esc-50/sample/rooster/1-27724-A.ogg', '../data/esc-50/sample/rooster/1-44831-A.ogg', '../data/esc-50/sample/rooster/1-26806-A.ogg', '../data/esc-50/sample/rooster/1-34119-A.ogg', '../data/esc-50/sample/rooster/1-34119-B.ogg']

In [27]:
mfccs, chroma, mel, contrast,tonnetz = extract_feature(samples_dict.get('insects')[0])
ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])

In [28]:
mfccs, chroma, mel, contrast,tonnetz = extract_feature(samples_dict.get('insects')[0])
ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
print len(ext_features)

193


In [29]:
features = np.empty((0,193))
print features.shape

(0, 193)


In [30]:
features = np.vstack([features,ext_features])
print features

[[ -2.49675622e+02   8.76930928e+01  -1.06365907e+01   1.25032669e+01
    4.64620967e+00   2.15535964e+01   1.29809755e+00  -5.20415561e-01
   -5.57909356e+00  -1.85501224e+00  -4.52347089e+00   9.25477340e+00
    4.83212888e-01   4.58398645e+00   1.68070856e+00   2.20859392e+00
   -2.17902442e+00   3.18389167e+00  -3.95563234e+00  -3.49785693e-01
   -6.19260976e+00  -1.10243040e+00  -4.46826041e+00   2.20117897e+00
   -1.12335099e+00   4.90120774e+00   7.22403137e-01   7.83006114e+00
    1.80591717e+00   8.66433927e+00   3.47956374e+00   9.57140115e+00
    3.74476504e+00   3.03680539e+00  -4.06008635e+00   1.72377973e+00
   -1.76241086e+00   5.00777948e+00   2.96484258e-01   1.93045239e+00
    7.59394045e-01   7.68509256e-01   7.32613646e-01   6.40876586e-01
    7.03065887e-01   6.32504914e-01   5.34453624e-01   5.66907861e-01
    6.05313013e-01   7.26594944e-01   7.60901096e-01   6.98334068e-01
    5.54438507e-01   2.92639076e-01   1.61382640e-01   8.67036091e-02
    8.27784347e-02  

In [46]:
features = np.empty((0,193))
for k, v in samples_dict.iteritems():
    if k not in ['insects', 'rooster']:
        continue
    for i in v:
        mfccs, chroma, mel, contrast,tonnetz = extract_feature(i)
        ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        features = np.vstack([features,ext_features])
features = np.array(features)

In [70]:
print features.shape
print features

(16, 193)
[[ -2.49675622e+02   8.76930928e+01  -1.06365907e+01 ...,   5.22720948e-02
   -2.61159677e-03   9.99713920e-03]
 [ -2.71216527e+01   8.10875418e+01  -2.14870425e+01 ...,  -8.82507703e-03
   -1.73585694e-03   1.56629750e-03]
 [ -3.07675876e+02   3.70427281e+01  -1.42486262e+01 ...,   2.51055671e-02
    6.43736371e-03  -1.64034112e-03]
 ..., 
 [ -4.15920721e+02   5.48410367e+01  -1.92863541e+01 ...,   8.46185754e-02
    8.84525074e-03  -8.10284288e-04]
 [ -5.64033295e+02   3.00150099e+01  -6.48551923e+00 ...,   3.75198051e-02
   -9.49510185e-03  -7.94706822e-04]
 [ -6.30568980e+02   3.42620522e+01  -6.55522122e+00 ...,  -1.06310258e-02
   -1.08761674e-02  -1.57340464e-02]]


In [43]:
labels = [0] * 8 + [1] * 8

In [44]:
print labels

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


In [50]:
one_hot = one_hot_encode(labels)

16


In [24]:
# training_epochs = 50
n_hidden_units_one = 280 
n_hidden_units_two = 300
# sd = 1 / np.sqrt(n_dim)
# learning_rate = 0.01

n_classes = 10
n_dim = tr_features.shape[1]

print tr_features.shape
print ts_features.shape

(1761, 193)
(925, 193)


In [25]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

In [26]:
model = Sequential()
model.add(Dense(n_hidden_units_one, input_dim=n_dim, kernel_initializer='uniform', activation='relu'))
model.add(Dense(n_hidden_units_two, kernel_initializer='uniform', activation='relu'))
model.add(Dense(n_classes, kernel_initializer='uniform', activation='softmax'))

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.fit(tr_features, tr_labels, nb_epoch=10, batch_size=10)

Epoch 1/10
1761/1761 [==============================] - 0s - loss: 1.7359 - acc: 0.7479     
Epoch 2/10
1761/1761 [==============================] - 0s - loss: 0.5630 - acc: 0.8836     
Epoch 3/10
1761/1761 [==============================] - 0s - loss: 0.3883 - acc: 0.9250     
Epoch 4/10
1761/1761 [==============================] - 0s - loss: 0.3160 - acc: 0.9438     
Epoch 5/10
1761/1761 [==============================] - 0s - loss: 0.2962 - acc: 0.9597     
Epoch 6/10
1761/1761 [==============================] - 0s - loss: 0.1943 - acc: 0.9756     
Epoch 7/10
1761/1761 [==============================] - 0s - loss: 0.2443 - acc: 0.9642     
Epoch 8/10
1761/1761 [==============================] - 0s - loss: 0.1548 - acc: 0.9779     
Epoch 9/10
1761/1761 [==============================] - 0s - loss: 0.0858 - acc: 0.9903     
Epoch 10/10
1761/1761 [==============================] - 0s - loss: 0.1493 - acc: 0.9648     


In [37]:
scores = model.evaluate(ts_features, ts_labels)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/925 [>.............................] - ETA: 0sacc: 36.86%


In [38]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_r = pca.fit(tr_features).transform(tr_features)

In [39]:
print X_r

[[ -41.38544709  -11.36271457]
 [ -50.95878037   38.3800235 ]
 [ -23.5522534   -99.7148314 ]
 ..., 
 [  36.54023902 -162.42266445]
 [   4.71517831 -212.38315289]
 [ -45.05800724    3.42315611]]
